In [1]:
from zarr_tools import convert
import nd2
import os
import dask.array as da
from glob import glob
from collections import OrderedDict
import asyncio
import numpy as np

In [4]:
paths = glob("/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/*/*/TRITC*.nd2")
paths

['/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230427-ecoli/set1Tetra/TRITC-3D.nd2',
 '/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230425-ecoli/NoAB/TRITC-3D-20min.nd2',
 '/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230425-ecoli/set1NoAB/TRITC-3D-20min.nd2',
 '/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230426-ecoli/set2CHP/TRITC.nd2',
 '/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230426-ecoli/set2CHP/TRITC-3D.nd2',
 '/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230426-ecoli/set1Genta/TRITC-24H.nd2',
 '/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230426-ecoli/set1Genta/TRITC-3D.nd2']

In [5]:
async def make_projection(nd2_path:str, axis_name:str="Z", op="mean"):
    try:
        print(nd2_path)
        d = nd2.ND2File(nd2_path)
        print(d.sizes) 
        axes = list(d.sizes)
        axis = axes.index(axis_name)
        dtype=np.uint16 if op=="max" else np.float32
        proj = d.to_dask().__getattribute__(op)(axis=axis, dtype=dtype)
        new_axes = axes.pop(axis)
        new_sizes = {a: d.sizes[a] for a in axes}
        print(new_sizes)
        save_path = nd2_path.replace(".nd2", f"-{op}{axis_name}.zarr")
        assert not os.path.exists(save_path)
        convert.to_zarr(proj, path=save_path, steps=4, 
                        channel_axis=None, 
                        name="TRITC_meanZ",
                        sizes=new_sizes,
                        colormap="green",
                        lut=[400,600],
                       )
        
    except OSError:
        d = None
        print(f"Unamble to read {nd2_path}")
    except AssertionError:
        print(f"File Exists! {save_path}")


In [6]:
asyncio.gather(*[make_projection(path) for path in paths])

<_GatheringFuture pending>

/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230427-ecoli/set1Tetra/TRITC-3D.nd2
{'T': 25, 'P': 2, 'Z': 25, 'Y': 6752, 'X': 21756}
{'T': 25, 'P': 2, 'Y': 6752, 'X': 21756}
/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230427-ecoli/set1Tetra/TRITC-3D-meanZ.zarr
(1, 1, 1688, 7252) (25, 2, 6752, 21756) 600
saved /home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230427-ecoli/set1Tetra/TRITC-3D-meanZ.zarr/0
(1, 1, 844, 3626) (25, 2, 3376, 10878) 600
saved /home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230427-ecoli/set1Tetra/TRITC-3D-meanZ.zarr/1
(2, 2, 1266, 5439) (25, 2, 1688, 5439) 26
saved /home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230427-ecoli/set1Tetra/TRITC-3D-meanZ.zarr/2
(6, 2, 844, 2720) (25, 2, 844, 2720) 5
saved /home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230427-ecoli/set1Tetra/TRITC-3D-meanZ.zarr/3
/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/20230425-ecoli/NoAB/TRITC-3D-20min.nd2
{'T': 16, 'P': 

In [ ]:
glob("/home/aaristov/Multicell/Lena/new_data/E.coli/Timelapse/*/*/meanZ.zarr")